In [174]:
import pandas as pd
import numpy as np

# IDEA
- rolling window prediction
- create some interactive thing per state
- rainfall happened one day, like a lot, likelihood of flood happening, in next few days
- rainfall happened over a period of days, likelihood of flood happening



In [175]:
df_flood = pd.read_csv('../data/Flood Events/floodDataProcessed.csv')
df_rainfall = pd.read_csv('../data/Weather Data/daily-rainfall-at-state-level.csv')

In [176]:
# analysis dates will be from 2009 to 2019

df_flood["year"] = df_flood["Start Date"].str.extract(r"(\d{4})")
df_flood["year"] = pd.to_datetime(df_flood["year"], format="%Y")

mask = df_flood["year"].dt.year >= 2009
df_flood = df_flood.loc[mask]

df_flood.sort_values(by='year', inplace=True)
df_flood.dropna(inplace=True)
df_flood.shape

(624, 6)

In [177]:
df_rainfall['date'] = pd.to_datetime(df_rainfall['date'], format='%Y-%m-%d')
df_rainfall['year'] = df_rainfall['date'].dt.year
df_rainfall = df_rainfall[(df_rainfall['year'] >= 2009) & (df_rainfall['year'] <= 2019)]
df_rainfall.shape

(144612, 9)

In [178]:
df_flood

,Unnamed: 0,Start Date,End Date,Duration(Days),State,year
951,951,2009-11-03,2009-11-08,5,TAMIL NADU,2009-01-01
948,948,/07/2009,/09/2009,Diffing the dates,ASSAM,2009-01-01
949,949,2009-09-25,2009-10-12,17,KARNATAKA,2009-01-01
950,950,2009-10-09,2009-10-09,0,MEGHALAYA,2009-01-01
641,641,2009-03-11,2009-08-11,153,TAMIL NADU,2009-01-01
...,...,...,...,...,...,...
719,719,2019-01-07,2019-05-07,120,MAHARASHTRA,2019-01-01
718,718,2019-06-27,2019-07-13,16,ASSAM,2019-01-01
1025,1025,2019-10-19,2019-10-23,4,KARNATAKA,2019-01-01
725,725,2019-11-30,2019-06-12,Diffing the dates,TAMIL NADU,2019-01-01


In [ ]:
# Data cleaning and transformation steps

# 1) Parse dates; coerce bad strings (e.g., "Diffing the dates") to NaT
for col in ['Start Date', 'End Date']:
    df_flood[col] = pd.to_datetime(df_flood[col], errors='coerce')

# 2) Duration numeric (for repairing missing/invalid End Date)
df_flood['Duration(Days)'] = pd.to_numeric(df_flood['Duration(Days)'], errors='coerce')

# 3) If End Date is missing but we have Start Date + Duration, compute End = Start + (Duration-1) days
mask_fix = df_flood['End Date'].isna() & df_flood['Start Date'].notna() & df_flood['Duration(Days)'].notna()
df_flood.loc[mask_fix, 'End Date'] = df_flood.loc[mask_fix, 'Start Date'] + pd.to_timedelta(df_flood.loc[mask_fix, 'Duration(Days)'] - 1, unit='D')

# 4) Drop rows still missing critical fields
df_flood = df_flood.dropna(subset=['Start Date', 'End Date', 'State'])

# 5) If End < Start (e.g., data entry error), swap them
swap = df_flood['End Date'] < df_flood['Start Date']
df_flood.loc[swap, ['Start Date','End Date']] = df_flood.loc[swap, ['End Date','Start Date']].values

# 6) Clamp to the analysis window
start_all = pd.Timestamp('2009-01-01')
end_all   = pd.Timestamp('2019-12-31')
df_flood['Start Date'] = df_flood['Start Date'].clip(lower=start_all, upper=end_all)
df_flood['End Date']   = df_flood['End Date'].clip(lower=start_all, upper=end_all)

# 7) Build per-row date ranges and explode to daily rows for flooding periods
ranges = (df_flood
    .assign(date=df_flood.apply(lambda r: pd.date_range(r['Start Date'], r['End Date']), axis=1))
    .explode('date')[['State','date']]
    .drop_duplicates()
    .assign(flood=1)
)

# 8) Create full State × Date grid (all days, all states)
states = df_flood['State'].dropna().unique()
all_dates = pd.date_range(start_all, end_all, freq='D')
full = pd.MultiIndex.from_product([states, all_dates], names=['State','date']).to_frame(index=False)

# 9) Left-join flooding days onto the full grid; fill others with 0
out = full.merge(ranges, on=['State','date'], how='left')
out['flood'] = out['flood'].fillna(0).astype('int8')

# columns ['State','date','flood']

panel = out.pivot(index='date', columns='State', values='flood').fillna(0).astype('int8')
days_by_state = out.groupby('State')['flood'].sum()

print(out.head())


        State       date  flood
0  TAMIL NADU 2009-01-01      0
1  TAMIL NADU 2009-01-02      0
2  TAMIL NADU 2009-01-03      0
3  TAMIL NADU 2009-01-04      0
4  TAMIL NADU 2009-01-05      0


C:\Users\bekim\AppData\Local\Temp\ipykernel_329440\3731402596.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_flood['Start Date'] = df_flood['Start Date'].clip(lower=start_all, upper=end_all)
C:\Users\bekim\AppData\Local\Temp\ipykernel_329440\3731402596.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_flood['End Date']   = df_flood['End Date'].clip(lower=start_all, upper=end_all)


In [180]:
df_flood_2 = out.copy()

df_flood_2[df_flood_2['flood'] == 1]

,State,date,flood
69,TAMIL NADU,2009-03-11,1
70,TAMIL NADU,2009-03-12,1
71,TAMIL NADU,2009-03-13,1
72,TAMIL NADU,2009-03-14,1
73,TAMIL NADU,2009-03-15,1
...,...,...,...
122969,CHATTISGARH,2015-09-26,1
122970,CHATTISGARH,2015-09-27,1
122971,CHATTISGARH,2015-09-28,1
122972,CHATTISGARH,2015-09-29,1


In [181]:
states = df_flood_2['State'].dropna().unique()
states.sort()
states

array(['ANDAMAN & NICOBAR ISLANDS', 'ANDHRA  PRADESH', 'ANDHRA PRADESH',
       'ARUNACHAL PRADESH', 'ASSAM', 'BIHAR', 'CHATTISGARH',
       'CHHATTISGARH', 'GUJARAT', 'HIMACHAL PRADESH', 'JAMMU & KASHMIR',
       'JHARKHAND', 'KARNATAKA', 'KERALA', 'MADHYA PRADESH',
       'MAHARASHTRA', 'MEGHALAYA', 'MIZORAM', 'NAGALAND', 'NEW DELHI',
       'ODISHA', 'PUDUCHERRY', 'PUNJAB', 'RAJASTHAN', 'SIKKIM',
       'TAMIL NADU', 'TELANGANA', 'TRIPURA', 'UTTAR PRADESH',
       'UTTARAKHAND', 'WEST BENGAL'], dtype=object)

In [193]:
def check_state(name):
    for state in states:
        if state.lower().strip() in str(name).lower().strip():
            return state
    return np.nan

df_rainfall['state_clean'] = df_rainfall['state_name'].apply(check_state)

df_rainfall_2 = df_rainfall.dropna(subset=["state_clean"]).reset_index(drop=True)

In [194]:
df_rainfall_2['state_name'].unique()


array(['UTTARAKHAND', 'ASSAM', 'TRIPURA', 'TELANGANA', 'HIMACHAL PRADESH',
       'MEGHALAYA', 'MAHARASHTRA', 'PUNJAB', 'JHARKHAND',
       'ANDHRA PRADESH', 'ARUNACHAL PRADESH', 'UTTAR PRADESH',
       'MADHYA PRADESH', 'KARNATAKA', 'WEST BENGAL', 'RAJASTHAN',
       'CHHATTISGARH', 'NAGALAND', 'BIHAR', 'PUDUCHERRY', 'GUJARAT',
       'ODISHA', 'TAMIL NADU', 'KERALA', 'SIKKIM', 'MIZORAM'],
      dtype=object)

In [196]:
df_rainfall_2.drop(columns=['state_name', 'year'], inplace=True)

In [198]:
merge_df = df_rainfall_2.merge(df_flood_2, left_on=['state_clean', 'date'], right_on=['State', 'date'], how='left')

In [202]:
merge_df.to_csv('../data/merged_flood_rainfall.csv', index=False)